# Flyweight (again)

> Dealing with more complex scenarios

In this lesson we are going to look into ways of implementing text formatting.

Let's begin with a simple `FormattedText` class that stores the plain text it's trying to format. For the sake of simplicity, this class will only deal with capitalizing certaing letters of the text, so we will provide a method to do just that and a string representation for the class.

In order to store out formatting information, we will use a brute force approach: we will have a boolean array of the same length as the plain text and for each letter that we want to capitalize, we will set the matching index in the boolean array to `True`.

In [1]:
class FormattedText:
    def __init__(self, plain_text):
        self.plain_text = plain_text
        self.caps = [False] * len(plain_text)

    def capitalize(self, start, end):
        for i in range(start, end):
            self.caps[i] = True

    def __str__(self):
        result = []
        for i in range(len(self.plain_text)):
            c = self.plain_text[i]
            result.append(c.upper() if self.caps[i] else c)
        return ''.join(result)

Let's test our class:

In [2]:
ft = FormattedText('This is a brave new world')
ft.capitalize(10, 15)
print(ft)
print(len(ft.plain_text))
print(len(ft.caps))

This is a BRAVE new world
25
25


Our code works, but we're wasting too much memory with our boolean array: if we were to store a very long text (like, for example, a complete novel), our boolean array would be equally long and possibly most of the data within would be wasted because we probably won't be capitalizing most letters.

Let's create instead a new `BetterFormattedText` class that implements the **flyweight pattern** to optimize memory usage: our class will store the plain text and an empty formatting list; we will also create an inner `TextRange` flyweight class that will specify the range of characters that we want to format as well as formatting options.

In [3]:
class BetterFormattedText:
    def __init__(self, plain_text):
        self.plain_text = plain_text
        self.formatting = []

    class TextRange:
        def __init__(self, start, end, capitalize=False, bold=False, italic=False):
            """We specify what kinf of formatting we are doing as well as the range (start and end positions)"""
            self.start = start
            self.end = end
            self.bold = bold
            self.capitalize = capitalize
            self.italic = italic

        def covers(self, position):
            """Utility method that returns True if position is within the range of the formatting"""
            return self.start <= position <= self.end

    def get_range(self, start, end):
        """We create a TextRange instance and append it to our formatting list"""
        range = self.TextRange(start, end)
        self.formatting.append(range)
        return range

    def __str__(self):
        """We go through our plain text and apply the formatting. For simplicity, we will only deal with capitalization"""
        result = []
        for i in range(len(self.plain_text)):
            c = self.plain_text[i]
            for r in self.formatting:
                if r.covers(i) and r.capitalize:
                    c = c.upper()
            result.append(c)
        return ''.join(result)

Instead of wasting space like in our first implementation, our new class has a much smaller list that simply stores ranges, and the API that the class provides allows us to easily add formatting ranges to the text.

Let's see our new class in action:

In [4]:
bft = BetterFormattedText('This is a brave new world')
bft.get_range(16, 19).capitalize = True
print(bft)
print(len(bft.plain_text))
print(len(bft.formatting))

This is a brave NEW world
25
1
